In [1]:
import sys
import time
import os
import glob
from time import sleep
import datetime
import pytz
import argparse

import numpy as np
import pandas as pd
import pickle
import joblib

import scipy
import sklearn 
from sklearn.metrics import accuracy_score, classification_report

from scipy.stats import skew, kurtosis

savepath= 'D:/꿩/대학교/3/코옵/★기계시설물 고장 예지 센서/savepath/'
filepath='D:/꿩/대학교/3/코옵/★기계시설물 고장 예지 센서/변형/'

In [2]:
data = pd.read_csv('D:/꿩/대학교/3/코옵/★기계시설물 고장 예지 센서/변형/2.2kW_L-DSF-01_training_with_label.csv')

In [3]:
data = data.drop(['Date'], axis = 1)
data = data.drop(['WATT'], axis = 1)

In [4]:
test_data=pd.read_csv(filepath+'2.2kW_cur_L-DSF-01_validation_with_label.csv')

In [5]:
test_data

,Date,R_AbsMax,S_AbsMax,T_AbsMax,R_AbsMean,S_AbsMean,T_AbsMean,R_P2P,S_P2P,T_P2P,...,S_2x,T_2x,R_3x,S_3x,T_3x,R_4x,S_4x,T_4x,WATT,LABEL
0,2020-11-03 13:13:05,3.403320,3.083008,4.003906,2.068058,1.881215,2.207894,6.766602,6.045898,7.407227,...,3.172200,3.672384,0.014972,0.016815,0.016825,0.023058,0.029118,0.027701,2.2,0
1,2020-11-06 13:13:35,3.283203,3.042969,4.003906,2.045135,1.851146,2.218224,6.566406,5.925781,7.447266,...,3.018571,3.573038,0.017420,0.012482,0.015383,0.024391,0.022549,0.024005,2.2,0
2,2020-11-07 01:43:06,0.200195,0.200195,0.400391,0.000601,0.012032,0.307140,0.280273,0.240234,0.200195,...,0.003155,0.002524,0.001310,0.003527,0.002177,0.001299,0.002767,0.002863,2.2,0
3,2020-11-07 01:43:06,0.160156,0.240234,0.400391,0.000921,0.011771,0.306799,0.280273,0.280273,0.240234,...,0.001736,0.003016,0.001227,0.002740,0.002817,0.001124,0.003745,0.004328,2.2,0
4,2020-11-07 03:13:11,3.363281,3.163086,4.124023,2.028139,1.841477,2.231237,6.686523,6.206055,7.687500,...,3.100697,3.696719,0.021505,0.020882,0.026651,0.026565,0.024807,0.023144,2.2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4594,2020-11-16 08:17:30,3.283203,3.083008,3.963867,2.069299,1.881636,2.246712,6.526367,6.005859,7.327148,...,3.162185,3.724256,0.055533,0.050859,0.059263,0.063400,0.060408,0.065105,2.2,3
4595,2020-11-16 08:17:30,3.363281,3.083008,3.923828,2.079108,1.890825,2.256802,6.646484,6.045898,7.327148,...,3.125027,3.675339,0.057672,0.055412,0.059837,0.113304,0.106505,0.114273,2.2,3
4596,2020-11-16 08:20:56,3.403320,3.083008,4.003906,2.079008,1.890564,2.263729,6.686523,6.125977,7.527344,...,3.211975,3.792254,0.019447,0.017908,0.020094,0.035279,0.031686,0.027314,2.2,3
4597,2020-11-16 08:20:56,3.403320,3.123047,4.003906,2.072562,1.887181,2.253679,6.726562,6.125977,7.487305,...,3.217054,3.791530,0.055852,0.050406,0.053594,0.029687,0.027959,0.028419,2.2,3


In [6]:
data

,R_AbsMax,S_AbsMax,T_AbsMax,R_AbsMean,S_AbsMean,T_AbsMean,R_P2P,S_P2P,T_P2P,R_RMS,...,R_2x,S_2x,T_2x,R_3x,S_3x,T_3x,R_4x,S_4x,T_4x,LABEL
0,3.403320,3.123047,4.043945,2.057587,1.889543,2.245951,6.726562,6.166016,7.607422,2.296682,...,3.282771,3.058104,3.579351,0.022117,0.019693,0.023455,0.031669,0.027953,0.024256,0
1,3.283203,3.083008,4.003906,2.059709,1.883498,2.254960,6.526367,6.005859,7.487305,2.287841,...,3.433224,3.183656,3.761500,0.013864,0.011091,0.014099,0.030992,0.039196,0.035626,0
2,3.283203,3.083008,3.963867,2.059209,1.884539,2.255280,6.526367,6.085938,7.407227,2.288444,...,3.432319,3.184894,3.755824,0.022333,0.019714,0.020837,0.024400,0.026358,0.023356,0
3,3.283203,3.123047,3.963867,2.065936,1.881936,2.254640,6.526367,6.085938,7.407227,2.295448,...,3.448775,3.188166,3.774974,0.020763,0.018589,0.020160,0.021301,0.024387,0.022983,0
4,3.363281,3.163086,3.963867,2.063974,1.901255,2.222268,6.686523,6.206055,7.407227,2.293941,...,3.317691,3.091339,3.568165,0.012395,0.012090,0.013433,0.016602,0.018446,0.015287,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36763,3.363281,3.163086,4.003906,2.084033,1.875850,2.275960,6.686523,6.166016,7.527344,2.334815,...,3.407100,3.087204,3.727322,0.052801,0.051043,0.052604,0.072128,0.065349,0.071531,3
36764,3.363281,3.163086,4.043945,2.093102,1.889503,2.289674,6.686523,6.286133,7.607422,2.346897,...,3.442630,3.130037,3.764798,0.022162,0.021225,0.021776,0.023548,0.023365,0.023047,3
36765,3.363281,3.163086,4.043945,2.088117,1.887081,2.282687,6.686523,6.206055,7.527344,2.339909,...,3.442938,3.132089,3.758024,0.048946,0.045703,0.047815,0.075257,0.066345,0.073342,3
36766,3.363281,3.203125,4.043945,2.083112,1.875770,2.266111,6.686523,6.246094,7.567383,2.341179,...,3.497469,3.169280,3.802370,0.015485,0.015876,0.012912,0.025546,0.029880,0.029603,3


In [7]:
corr = data.corr()

In [8]:
corr

,R_AbsMax,S_AbsMax,T_AbsMax,R_AbsMean,S_AbsMean,T_AbsMean,R_P2P,S_P2P,T_P2P,R_RMS,...,R_2x,S_2x,T_2x,R_3x,S_3x,T_3x,R_4x,S_4x,T_4x,LABEL
R_AbsMax,1.000000,0.976039,0.951810,0.595678,0.590620,0.560857,0.986013,0.979231,0.880473,0.656366,...,0.284837,0.283625,0.270297,0.389041,0.405073,0.316112,0.193779,0.208885,0.146508,0.151402
S_AbsMax,0.976039,1.000000,0.944998,0.557187,0.567891,0.533517,0.955159,0.992243,0.858993,0.621417,...,0.271015,0.277088,0.261770,0.396442,0.415429,0.320316,0.190434,0.209180,0.143383,0.140357
T_AbsMax,0.951810,0.944998,1.000000,0.667154,0.658479,0.683214,0.952813,0.955553,0.954192,0.727794,...,0.329955,0.329415,0.339922,0.344419,0.358639,0.281676,0.185279,0.198350,0.145601,0.242925
R_AbsMean,0.595678,0.557187,0.667154,1.000000,0.987461,0.974858,0.707547,0.626479,0.833828,0.993344,...,0.493320,0.488791,0.487994,0.031974,0.027373,0.044086,0.074317,0.068978,0.082227,0.366145
S_AbsMean,0.590620,0.567891,0.658479,0.987461,1.000000,0.962475,0.701358,0.636555,0.826340,0.977272,...,0.484093,0.491775,0.480110,0.033976,0.030720,0.046690,0.075727,0.070231,0.083155,0.317210
T_AbsMean,0.560857,0.533517,0.683214,0.974858,0.962475,1.000000,0.667867,0.602308,0.848339,0.972359,...,0.484260,0.481748,0.500837,0.019666,0.014894,0.033541,0.076537,0.070728,0.085275,0.400825
R_P2P,0.986013,0.955159,0.952813,0.707547,0.701358,0.667867,1.000000,0.972371,0.926145,0.757719,...,0.338832,0.337275,0.323093,0.346177,0.358427,0.286073,0.182938,0.194735,0.144553,0.179566
S_P2P,0.979231,0.992243,0.955553,0.626479,0.636555,0.602308,0.972371,1.000000,0.895243,0.686242,...,0.305746,0.311667,0.296597,0.375616,0.392486,0.306340,0.190567,0.205146,0.145384,0.161350
T_P2P,0.880473,0.858993,0.954192,0.833828,0.826340,0.848339,0.926145,0.895243,1.000000,0.873079,...,0.407321,0.407836,0.419441,0.250594,0.256821,0.214992,0.161825,0.167019,0.138265,0.286322
R_RMS,0.656366,0.621417,0.727794,0.993344,0.977272,0.972359,0.757719,0.686242,0.873079,1.000000,...,0.497229,0.490785,0.492854,0.069924,0.068185,0.073341,0.088818,0.086268,0.090736,0.406111


In [9]:
#상관관계분석을 통해 바뀐 data를  저장
treat_train=data
treat_test=test_data
#training_with_label-->0.8개는 train 0.2개는 validation,
#validation_with_label-->test data로 바꿈

x_train = treat_train.sample(frac = 0.8)
x_valid = treat_train.drop(x_train.index)
x_test = treat_test

len(x_train) + len(x_valid) + len(x_test)

41367

In [10]:
y_train = x_train['LABEL']
y_test = x_test['LABEL']
y_valid = x_valid['LABEL']

x_train = x_train.drop(['LABEL'],axis = 1)
x_valid = x_valid.drop(['LABEL'],axis = 1)
x_test = x_test.drop(['LABEL'],axis = 1)

In [11]:
print("x_train.shape:", x_train.shape)
print("x_valid.shape:", x_valid.shape)
print("x_test.shape:", x_test.shape)
print("y_train.shape:", y_train.shape)
print("y_valid.shape:", y_valid.shape)
print("y_test.shape:", y_test.shape)

x_train.shape: (29414, 45)
x_valid.shape: (7354, 45)
x_test.shape: (4599, 47)
y_train.shape: (29414,)
y_valid.shape: (7354,)
y_test.shape: (4599,)


In [12]:
x_train_exp = np.expand_dims(x_train, -1)  
x_valid_exp = np.expand_dims(x_valid, -1)  
x_test_exp = np.expand_dims(x_test, -1)

print("x_train_exp의 형태:", x_train_exp.shape)
print("x_valid_exp의 형태:", x_valid_exp.shape)
print("x_test_exp의 형태:", x_test_exp.shape)

x_train_exp의 형태: (29414, 45, 1)
x_valid_exp의 형태: (7354, 45, 1)
x_test_exp의 형태: (4599, 47, 1)


In [13]:
y_train[y_train == 0] = 1
y_valid[y_valid == 0] = 1
y_test[y_test == 0] = 1

y_train[y_train == 3] = 0
y_valid[y_valid == 3] = 0
y_test[y_test == 3] = 0

<ipython-input-13-48e1413a8b22>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test[y_test == 0] = 1
<ipython-input-13-48e1413a8b22>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_test[y_test == 3] = 0


In [14]:
#상관관계분석, 아직 안해봄!

import matplotlib.cm as cm
from matplotlib.collections import EllipseCollection

df = pd.DataFrame(data = x_train, 
                  columns= ["sensor_{}".format(label+1) for label in range(x_train.shape[1])])

data = df.corr()

def plot_corr_ellipses(data, ax =None, **kwargs):

    M = np.array(data)
    if not M.ndim == 2:
        raise ValueError('data must be a 2D array')
    if ax is None:
        fig, ax = plt.subplots(1, 1, subplot_kw={'aspect':'equal'})
        ax.set_xlim(-0.5, M.shape[1] - 0.5)
        ax.set_ylim(-0.5, M.shape[0] - 0.5)

    xy = np.indices(M.shape)[::-1].reshape(2, -1).T

    w = np.ones_like(M).ravel()
    h = 1 - np.abs(M).ravel()
    a = 45 * np.sign(M).ravel()

    ec = EllipseCollection(widths=w, heights=h, angles=a, units='x', offsets=xy,
                           transOffset=ax.transData, array=M.ravel(), **kwargs)
    ax.add_collection(ec)

    if isinstance(data, pd.DataFrame):
        ax.set_xticks(np.arange(M.shape[1]))
        ax.set_xticklabels(data.columns, rotation=90)
        ax.set_yticks(np.arange(M.shape[0]))
        ax.set_yticklabels(data.index)

    return ec


fig, ax = plt.subplots(1, 1, figsize=(20, 20))
cmap = cm.get_cmap('jet', 31)
m = plot_corr_ellipses(data, ax=ax, cmap=cmap)
cb = fig.colorbar(m)
cb.set_label('Correlation coefficient')
# ax.margins(0.1)

plt.title('Correlation between Feature')
# labels = label
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
plt.tight_layout()
plt.savefig(save_path + 'corr.png', dpi=100, bbox_inches='tight')  # 그림 저장

plt.show()

NameError: name 'plt' is not defined

In [15]:
!pip install xgboost
from xgboost import XGBClassifier

"""
# 반드시 튜닝해야할 파라미터는  min_child_weight / max_depth / gamma
"""

xgb = XGBClassifier(
    learning_rate=0.1, 
    n_estimators=500, 
    max_depth=5,
    min_child_weight=3, 
    gamma=0.2, 
    subsample=0.6, 
    colsample_bytree=1.0,
    objective='binary:logistic', 
    nthread=4, 
    scale_pos_weight=1, 
    seed=27)

## XGBoost 학습

In [ ]:
from sklearn.model_selection import GridSearchCV

# grid search를 하고 싶을 경우 아래 Code를 주석 해제

parameters = {
    "learning_rate" : [0.1, 0.01, 0.001, 0.0001],
   # "eta"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
    "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
      "min_child_weight" : [ 1, 3, 5, 7 ],
      "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4],
      "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
      }

grid = GridSearchCV(xgb,
                    parameters, n_jobs=4,
                    scoring="neg_log_loss",
                    cv=5)
grid.fit(
    x_train.squeeze(), 
    y_train, 
    eval_metric="auc", 
    eval_set=[(x_train, y_train), (x_valid, y_valid)], 
    verbose=True)

##여기까지 실행했는데 계속 500epoch가 반복되고 코드가 끝나지 않아서 뒤에는 실행을 못햇습니다.

In [9]:
#XGboost평가

from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

y_pred = xgb.predict(x_test)
y_pred_proba = xgb.predict_proba(x_test)[:, 1]

print("\nAbout xgboost model")
print("Accuracy : %.4g" % accuracy_score(y_test, y_pred))
print("AUC Score (training set): %f" % roc_auc_score(y_test, y_pred_proba))
print("F1 Score (training set): %f" % f1_score(y_test, y_pred))

NameError: name 'x_test' is not defined

# XGboost 평가

In [4]:
draw_confusion_matrix(xgb, x_test, y_test, "xgboost")

NameError: name 'draw_confusion_matrix' is not defined

In [ ]:
draw_roc(xgb, x_test, y_test, "xgboost")

In [ ]:
xgb_results = xgb.evals_result()  # xgboost 모델의 평가 결과 불러오기
epochs = len(xgb_results['validation_0']['error'])    # iteration 수
x_axis = range(0, epochs)  # x축(epoch) 범위 설정


# plot classification error
fig, ax = plt.subplots(figsize=(10,4))
ax.plot(x_axis, xgb_results['validation_0']['error'], label='Train')
ax.plot(x_axis, xgb_results['validation_1']['error'], label='Validation')
ax.legend()

plt.ylabel('Classification Error')
plt.title('XGBoost Classification Error')
plt.tight_layout()
plt.savefig(save_path + 'learning_curve_error_xgb.png', dpi=100, bbox_inches='tight')
plt.show()


# plot auc
fig, ax = plt.subplots(figsize=(10,4))
ax.plot(x_axis, xgb_results['validation_0']['auc'], label='Train')
ax.plot(x_axis, xgb_results['validation_1']['auc'], label='Validation')
ax.legend()

plt.ylabel('AUC')
plt.title('XGBoost AUC')
plt.tight_layout()
plt.savefig(save_path + 'learning_curve_auc_xgb.png', dpi=100, bbox_inches='tight')
plt.show()

In [ ]:
#feature importance
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score

feat_imp = xgb.feature_importances_
idx = np.where(feat_imp > 0.004)
feat_imp_important = feat_imp[idx]
feat = ["sensor_{}".format(i+1) for i in idx[0]]
# clf.best_estimator_.booster().get_fscore()
res_df = pd.DataFrame({'Features': feat, 'Importance': feat_imp_important}).sort_values(by='Importance', ascending=False)
res_df.plot('Features', 'Importance', kind='bar', title='Feature Importances', figsize = (7, 5) )
plt.ylabel('Feature Importance Score')
plt.tight_layout()
plt.savefig(save_path + 'xgb_feature_importance.png', dpi=100, bbox_inches='tight')  # 그림 저장
plt.show()
print(res_df)
print(res_df["Features"].tolist())